### Imports

In [1]:
import random
import torch
import torch.nn.parallel
import torch.optim as optim
import torch.nn as nn
import torch.utils.data
from localbot_localization.src.models.pointnet import PointNet, feature_transform_regularizer
from localbot_localization.src.dataset import LocalBotDataset
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

torch.Size([100000, 3])


### Load the dataset

In [2]:
train_dataset = LocalBotDataset(seq=110, npoints=2000)
test_dataset = LocalBotDataset(seq=111, npoints=2000)
batch_size = 2

### Pytorch data loader

In [3]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=2)
# Num_workers tells the data loader instance how many sub-processes to use for data loading. If the num_worker is zero (default) the GPU has to wait for CPU to load data.

In [4]:
print(len(train_dataset), len(test_dataset))

92 23


### Build the model

In [5]:
feature_transform = False # lets use feature transform
model = PointNet(feature_transform=feature_transform)


### Optimizer and Loss

In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.001) # the most common optimizer in DL
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5) # variable learning rate. After 5 epochs, the lr decays 0.5
criterion = nn.MSELoss() # TODO: search for a better loss function!! Should we predict the translaction and rotation separately?? RESEARCH!

In [7]:
model.cuda() # move all model parameters to the GPU

PointNet(
  (feat): PointNetfeat(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2

In [8]:
num_batch = len(train_dataset) / batch_size

### Training loop

In [10]:
n_epochs = 40
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for epoch in range(n_epochs):
    t0 = datetime.now()
    scheduler.step() # here we are telling the scheduler that: n_epochs += 1
    train_loss = []
    for i, data in enumerate(train_dataloader):
        points, target = data
        #points.shape --> 2,10000,3
        #target.shape --> 2,6
    
        points = points.transpose(2, 1) # 3xN which is what our network is expecting
        points, target = points.cuda(), target.cuda() # move data into GPU
        
        optimizer.zero_grad() # Clears the gradients of all optimized tensors (always needed in the beginning of the training loop)
        
        model = model.train() # Sets the module in training mode. For example, the dropout module can only be use in training mode.
        
        #print(points.shape)
        
        pred, trans, trans_feat = model(points) # our model outputs the pose, and the transformations used,
            
        loss = criterion(pred, target)
        
        if feature_transform:
            loss += feature_transform_regularizer(trans_feat) * 0.001 ## Regularization! --> Prevent overfitting by adding something to the cost function. The simpler the model the lower the cost function
        
        
        loss.backward() # Computes the gradient of current tensor w.r.t. graph leaves.
        optimizer.step() # Performs a single optimization step (parameter update).
        
        train_loss.append(loss.item())
    train_loss = np.mean(train_loss)
    
    test_loss=[]
    for i, data in enumerate(train_dataloader):
        points, target = data
        points = points.transpose(2, 1) # 3xN which is what our network is expecting
        points, target = points.cuda(), target.cuda() # move data into GPU
        model = model.eval() # Sets the module in evaluation mode.

        pred, _, _ = model(points)
        
        loss = criterion(pred, target)
        
        test_loss.append(loss.item())
    test_loss = np.mean(test_loss)

    # save losses
    train_losses[epoch] = train_loss
    test_losses[epoch] = test_loss
    
    dt = datetime.now() - t0
    print(f'epoch {epoch+1}/{n_epochs}, train_loss: {train_loss:.4f}, test_loss: {test_loss:.4f}, duration: {dt}')
        
        
        
    

torch.Size([20, 3, 4000])


RuntimeError: CUDA out of memory. Tried to allocate 314.00 MiB (GPU 0; 1.96 GiB total capacity; 682.33 MiB already allocated; 169.50 MiB free; 702.00 MiB reserved in total by PyTorch)

In [ ]:
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.show()

In [ ]:
torch.save(model.state_dict(), 'pointnet.pth')